In [52]:
#acuurcy of this model is 0.8588796877420043 nearly 85 percent 
import numpy as np 
import pandas as pd 
df = pd.read_csv('data1.csv')



In [53]:
#dropping the unwanted ones which are not reqquired to predict rainfall tom
df = df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date', 'RISK_MM'],axis=1)
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)


In [54]:
train_set = train_set.drop("RainToday", axis = 1)

In [55]:
#converting cateogrical to numerical
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [56]:
X = train_set.drop("RainTomorrow", axis = 1)
y = train_set["RainTomorrow"].copy()

In [57]:
#all these are inbuilt packages that i am using directly
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["WindGustDir", "WindDir9am", "WindDir3pm"])),
        ("imp", SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [58]:
num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["MinTemp", "MaxTemp", "Rainfall", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm",
                                              "Pressure9am", "Pressure3pm", "Temp9am", "Temp3pm"])),
        ("imp", SimpleImputer(missing_values=np.nan, strategy='mean')),
        ('scaler', StandardScaler()),
        ])
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [59]:
#converting categorical data into numerical data using dictonary method
X_train_prepared = preprocess_pipeline.fit_transform(X)
y_train_prepared = y.map({'Yes':1, 'No':0})

In [60]:
X_test = test_set.drop("RainTomorrow", axis = 1)
y_test = test_set["RainTomorrow"].copy()

X_test_prepared = preprocess_pipeline.fit_transform(X_test)
y_test_prepared = y_test.map({'Yes':1, 'No':0})

In [64]:
#ann model that we learnt in lecture 5 using keras
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD

n_cols =  X_train_prepared.shape[1]
target =  to_categorical(y_train_prepared)

def get_new_model():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape = (n_cols,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    return model
print("Testing model with adam optimizer")
model = get_new_model()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')
model.fit(X_train_prepared, target)

Testing model with adam optimizer
Epoch 1/1
113754/113754 [==============================] - 17s 145us/step - loss: 0.3620


In [65]:
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=2) 

# Without adding any nodes or layers
model = get_new_model()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_prepared, target, validation_split=0.3, epochs=20, callbacks = [early_stopping_monitor])


Train on 79627 samples, validate on 34127 samples
Epoch 1/20
79627/79627 [==============================] - 14s 170us/step - loss: 0.3676 - acc: 0.8411 - val_loss: 0.3505 - val_acc: 0.8465
Epoch 2/20
79627/79627 [==============================] - 12s 148us/step - loss: 0.3518 - acc: 0.8482 - val_loss: 0.3450 - val_acc: 0.8506
Epoch 3/20
79627/79627 [==============================] - 12s 150us/step - loss: 0.3446 - acc: 0.8510 - val_loss: 0.3422 - val_acc: 0.8513
Epoch 4/20
79627/79627 [==============================] - 12s 149us/step - loss: 0.3400 - acc: 0.8528 - val_loss: 0.3445 - val_acc: 0.8511
Epoch 5/20
79627/79627 [==============================] - 12s 150us/step - loss: 0.3356 - acc: 0.8550 - val_loss: 0.3440 - val_acc: 0.8507


In [66]:
#we are increasing the no of layers to decrease the loss 
model = Sequential()
model.add(Dense(120, activation='relu', input_shape = (n_cols,)))
model.add(Dense(120, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_prepared, target, validation_split=0.3, epochs=20, callbacks = [early_stopping_monitor])

Train on 79627 samples, validate on 34127 samples
Epoch 1/20
79627/79627 [==============================] - 15s 185us/step - loss: 0.3667 - acc: 0.8412 - val_loss: 0.3528 - val_acc: 0.8460
Epoch 2/20
79627/79627 [==============================] - 12s 156us/step - loss: 0.3517 - acc: 0.8483 - val_loss: 0.3445 - val_acc: 0.8503
Epoch 3/20
79627/79627 [==============================] - 12s 156us/step - loss: 0.3450 - acc: 0.8505 - val_loss: 0.3424 - val_acc: 0.8523
Epoch 4/20
79627/79627 [==============================] - 12s 157us/step - loss: 0.3394 - acc: 0.8529 - val_loss: 0.3478 - val_acc: 0.8512
Epoch 5/20
79627/79627 [==============================] - 12s 153us/step - loss: 0.3347 - acc: 0.8546 - val_loss: 0.3429 - val_acc: 0.8533


In [67]:
#same as above
model = Sequential()
model.add(Dense(120, activation='relu', input_shape = (n_cols,)))
model.add(Dense(120, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_prepared, target, validation_split=0.3, epochs=20, callbacks = [early_stopping_monitor])

Train on 79627 samples, validate on 34127 samples
Epoch 1/20
79627/79627 [==============================] - 17s 210us/step - loss: 0.3672 - acc: 0.8410 - val_loss: 0.3485 - val_acc: 0.8489
Epoch 2/20
79627/79627 [==============================] - 14s 176us/step - loss: 0.3524 - acc: 0.8488 - val_loss: 0.3485 - val_acc: 0.8499
Epoch 3/20
79627/79627 [==============================] - 15s 192us/step - loss: 0.3455 - acc: 0.8505 - val_loss: 0.3433 - val_acc: 0.8514
Epoch 4/20
79627/79627 [==============================] - 15s 194us/step - loss: 0.3403 - acc: 0.8536 - val_loss: 0.3424 - val_acc: 0.8523
Epoch 5/20
79627/79627 [==============================] - 16s 196us/step - loss: 0.3351 - acc: 0.8557 - val_loss: 0.3440 - val_acc: 0.8534
Epoch 6/20
79627/79627 [==============================] - 15s 192us/step - loss: 0.3296 - acc: 0.8580 - val_loss: 0.3445 - val_acc: 0.8540


In [68]:
#training
model = Sequential()
model.add(Dense(120, activation='relu', input_shape = (n_cols,)))
model.add(Dense(120, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_prepared, target, validation_split=0.3, epochs=20, callbacks = [early_stopping_monitor])
#testing the data
test_loss, test_acc = model.evaluate(X_train_prepared, target)
print(test_acc)

Train on 79627 samples, validate on 34127 samples
Epoch 1/20
79627/79627 [==============================] - 16s 207us/step - loss: 0.3667 - acc: 0.8407 - val_loss: 0.3557 - val_acc: 0.8470
Epoch 2/20
79627/79627 [==============================] - 14s 178us/step - loss: 0.3521 - acc: 0.8474 - val_loss: 0.3458 - val_acc: 0.8497
Epoch 3/20
79627/79627 [==============================] - 14s 178us/step - loss: 0.3451 - acc: 0.8516 - val_loss: 0.3445 - val_acc: 0.8526
Epoch 4/20
79627/79627 [==============================] - 14s 178us/step - loss: 0.3393 - acc: 0.8538 - val_loss: 0.3472 - val_acc: 0.8508
Epoch 5/20
79627/79627 [==============================] - 14s 175us/step - loss: 0.3349 - acc: 0.8551 - val_loss: 0.3412 - val_acc: 0.8531
Epoch 6/20
79627/79627 [==============================] - 14s 176us/step - loss: 0.3300 - acc: 0.8579 - val_loss: 0.3441 - val_acc: 0.8517
Epoch 7/20
113754/113754 [==============================] - 7s 65us/step
0.8588796877420043
